In [66]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [67]:
input_size = 784 # 28x28
hidden_size1 = 10
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [68]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                            train=True,
                                       transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=transforms.ToTensor(),
                                           download=True)

In [69]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [70]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
     def __init__(self, input_size, hidden_size1, num_classes):
         super(NeuralNet, self).__init__()
         self.input_size = input_size
         self.l1 = nn.Linear(input_size, hidden_size1)
         self.relu1 = nn.ReLU()
         self.l2 = nn.Linear(hidden_size1, num_classes)

     def forward(self, x):
         out = self.l1(x)
         out = self.relu1(out)
         out = self.l2(out)
         # no activation and no softmax at the end
         return out

In [71]:
model = NeuralNet(input_size, hidden_size1, num_classes)

In [72]:
 # Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [73]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
     for i, (images, labels) in enumerate(train_loader):
         # origin shape: [100, 1, 28, 28]
         # resized: [100, 784]
         images = images.reshape(-1, 28*28)
         labels = labels
         # Forward pass
         outputs = model(images)
         loss = criterion(outputs, labels)
         # Backward and optimize
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         if (i+1) % 100 == 0:
             print (f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step[100/600], Loss: 1.1386
Epoch [1/2], Step[200/600], Loss: 0.5976
Epoch [1/2], Step[300/600], Loss: 0.4003
Epoch [1/2], Step[400/600], Loss: 0.4029
Epoch [1/2], Step[500/600], Loss: 0.5489
Epoch [1/2], Step[600/600], Loss: 0.3716
Epoch [2/2], Step[100/600], Loss: 0.3780
Epoch [2/2], Step[200/600], Loss: 0.2818
Epoch [2/2], Step[300/600], Loss: 0.3498
Epoch [2/2], Step[400/600], Loss: 0.1942
Epoch [2/2], Step[500/600], Loss: 0.4603
Epoch [2/2], Step[600/600], Loss: 0.3955


In [74]:
with torch.no_grad():
     n_correct = 0
     n_samples = 0
     for images, labels in test_loader:
         images = images.reshape(-1, 28*28)
         labels = labels
         outputs = model(images)
         # max returns (value ,index)
         _, predicted = torch.max(outputs.data, 1)
         n_samples += labels.size(0)
         n_correct += (predicted == labels).sum().item()
     acc = 100.0 * n_correct / n_samples
     print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 91.77 %
